In [1]:
import glob
import os
from tqdm import tqdm
import numpy as np

import sys
sys.path.append("/home/sho/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

In [2]:
sr = 16000
# result_base_dir = "/home/sho/Git/cuhksz-phd/notebooks/model_parameters/A26-A_audiofiles/"
result_base_dir = "./audio/240529_TAC/"
speakers = [str(v).zfill(4) for v in np.arange(11, 21)]
# ESD_dir = "/data/ESD/"

experiments = {
    "main": ["OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn", "OpenSMILE-OpenSMILE-WavLM_SER-globaloutput-GRL_esd_fcn", "relative-attributes_OpenSMILE_esd", "msemotts_OpenSMILE_esd"],
    "features": ["OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn", "WavLM_EPR-globaloutput-GRL_esd_fcn", "OpenSMILE_EPR-globaloutput-GRL_esd_fcn"],
    "methods": ["OpenSMILE_EPR-globaloutput-GRL_esd_fcn", "OpenSMILE_SER-globaloutput-GRL_esd_fcn", "relative-attributes_OpenSMILE_esd"],
    "GRL": ["OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn", "OpenSMILE-OpenSMILE-WavLM_SER-globaloutput-GRL_esd_fcn", "OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput_esd_fcn", "OpenSMILE-OpenSMILE-WavLM_SER-globaloutput_esd_fcn"],
    "SSL": [
        "OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn", "WavLM_EPR-globaloutput-GRL_esd_fcn",
        "OpenSMILE-OpenSMILE-cHubert_EPR-globaloutput-GRL_esd_fcn", "cHubert_EPR-globaloutput-GRL_esd_fcn",
        "OpenSMILE-OpenSMILE-emotion2vec_EPR-globaloutput-GRL_esd_fcn", "emotion2vec_EPR-globaloutput-GRL_esd_fcn",
           ],
}

import chardet
def openfile(txtfile):
    with open(txtfile, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(txtfile, "r", encoding=encoding) as f:
        content = f.readlines()
    return content
groundtruth_dir = {}
for speaker in tqdm(speakers[0:]):
    txtfile = f"/data/ESD/{speaker}/{speaker}.txt"
    content = openfile(txtfile)
    for c in content:
        try: 
            key, text, _ = c.split("\t")
        except ValueError:
            continue
        groundtruth_dir[key] = text

100%|██████████| 10/10 [00:02<00:00,  3.54it/s]


# MUSHRA
## Speech Naturalness

In [40]:
def init():
    whole_text = """var TestConfig = {
  "TestName": "Speech Naturalness Evaluation",
  "RateScalePng": "img/scale_abs.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

In [42]:
exid = "main"
num = 100000
start = 0

whole_text = init()
result_dir = result_base_dir + "expressiveness/"
modelnames = [os.path.basename(a) for a in glob.glob(result_dir + "*")]
modelnames.remove("reconstruct")
modelnames.sort()
filenames = glob.glob(result_dir + f"reconstruct/*.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    spk, name = basename.split("_")
    gt_path = result_base_dir + f"ground_truth/{basename}.wav"
    text = groundtruth_dir[basename]
    
    starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "NAT---{basename}",
    "Text": "{text}",
    "Files": {{
      "ground_truth": "{gt_path}",
      "reconstruct": "{path}",\n"""
    modelpaths = """"""
    for mn in modelnames:
        if not(mn in experiments[exid]):
            continue
        # print(mn)
        modelpath = result_dir + f"{mn}/{basename}-0.wav"
        added = f'      "{mn}": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Speech Naturalness Evaluation",
  "RateScalePng": "img/scale_abs.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "0011_000021",
    "TestID": "NAT---0011_000021",
    "Text": "As rich as Peter's son in law!",
    "Files": {
      "ground_truth": "./audio/240529_TAC/ground_truth/0011_000021.wav",
      "reconstruct": "./audio/240529_TAC/expressiveness/reconstruct/0011_000021.wav",
      "OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn": "./audio/240529_TAC/expressiveness/OpenSMILE-OpenSMILE-WavLM_EP

## Speech Similarity

In [43]:
def init():
    whole_text = """var TestConfig = {
  "TestName": "Speech Similarity Evaluation",
  "RateScalePng": "img/similar_scale.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

In [44]:
exid = "main"
num = 10000000
start = 0

whole_text = init()
result_dir = result_base_dir + "expressiveness/"
modelnames = [os.path.basename(a) for a in glob.glob(result_dir + "*")]
modelnames.remove("reconstruct")
modelnames.sort()
filenames = glob.glob(result_dir + f"reconstruct/*.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    spk, name = basename.split("_")
    
    starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "SIM---{basename}",
    "Files": {{
      "Reference": "{path}",\n"""
    modelpaths = """"""
    for mn in modelnames:
        if not(mn in experiments[exid]):
            continue
        # for r in range(2):
        modelpath = result_dir + f"{mn}/{basename}-0.wav"
        added = f'      "{mn}": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Speech Similarity Evaluation",
  "RateScalePng": "img/similar_scale.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "0011_000021",
    "TestID": "SIM---0011_000021",
    "Files": {
      "Reference": "./audio/240529_TAC/expressiveness/reconstruct/0011_000021.wav",
      "OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn": "./audio/240529_TAC/expressiveness/OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn/0011_000021-0.wav",
      "OpenSMILE-OpenSMILE-WavLM_SER-globaloutput-GRL_esd_fcn": "./audi

# BWS Test
## Utterance-level Emotion Controllability

In [3]:
import random

emos = ["Angry", "Happy", "Sad", "Surprise"]

def init():
    whole_text = """var TestConfig = {
  "TestName": "Utterance-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[\n"""
    return whole_text

evaluation_filenames = [
    # "0013_000731",
    "0014_000732",
    "0015_000726",
    # "0018_000037",
]

In [5]:
num = 10000000
start = 0
whole_text = init()
result_dir = result_base_dir + f"utterance_remain/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
filenames = glob.glob(f"{result_dir}{modelnames[0]}/*-Angry-0.0-0.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    basename, _, _ = basename.split("-")
    if not (basename in evaluation_filenames):
        continue
    spk, name = basename.split("_")
    for emotion in emos:
        for mn in modelnames:
            if "msemotts" in mn:
                continue
            starter = "  {\n"
            starter += f'    "Name": "{basename}-{emotion}",\n'
            starter += f'    "TestID": "{mn}---{basename}---{emotion}",\n'
            starter += f'    "Text": "Intended Emotion: <b>{emotion}</b>",\n'
            starter += f'    "Files": {{\n'
            modelpaths = """"""
            for itt, intensity in enumerate(intensities):
                modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
                added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
                modelpaths += added
            modelpaths += f'    }}\n'
            modelpaths += f'  }},\n'
            whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Utterance-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "0014_000732-Angry",
    "TestID": "OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn---0014_000732---Angry",
    "Text": "Intended Emotion: <b>Angry</b>",
    "Files": {
      "L": "./audio/240529_TAC/utterance_remain/OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.0-0.wav",
      "M": "./audio/240529_TAC/utterance_remain/OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.6-0.wav",
      "H": "./audio/240529_TAC/utterance_remain/OpenSMILE-OpenSMILE-WavLM_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-1.0-0.wav",
    }
  },
  {
    "Name": "0014_000732-Angry",
  

In [6]:
whole_text = init()
result_dir = result_base_dir + f"utterance_remain/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
test_arrays = []
for basename in evaluation_filenames:
    filenames = glob.glob(f"{result_dir}{modelnames[0]}/{basename}-Angry-0.0-0.wav")
    filenames.sort()
    for path in filenames[start:start+num]:
        spk, name = basename.split("_")
        for emotion in emos:
            for mn in modelnames:
                if "msemotts" in mn:
                    continue
                test_arrays += [[basename, emotion, mn]]
random.shuffle(test_arrays)
for test_array in test_arrays:
    basename, emotion, mn = test_array
    starter = "  {\n"
    starter += f'    "Name": "{basename}---{emotion}",\n'
    starter += f'    "TestID": "UC---{mn}---{basename}---{emotion}",\n'
    starter += f'    "Text": "Intended Emotion: <b>{emotion}</b>",\n'
    starter += f'    "Files": {{\n'
    modelpaths = """"""
    for itt, intensity in enumerate(intensities):
        modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
        added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},\n'
    whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Utterance-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "0014_000732---Angry",
    "TestID": "UC---OpenSMILE_SER-globaloutput-GRL_esd_fcn---0014_000732---Angry",
    "Text": "Intended Emotion: <b>Angry</b>",
    "Files": {
      "L": "./audio/240529_TAC/utterance_remain/OpenSMILE_SER-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.0-0.wav",
      "M": "./audio/240529_TAC/utterance_remain/OpenSMILE_SER-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.6-0.wav",
      "H": "./audio/240529_TAC/utterance_remain/OpenSMILE_SER-globaloutput-GRL_esd_fcn/0014_000732-Angry-1.0-0.wav",
    }
  },
  {
    "Name": "0014_000732---Sad",
    "TestID": "UC---OpenSMILE-OpenSMILE-WavLM_SER-globalout

## Word-level Emotion Controllability

In [49]:
worddir_paths = {}
filenames = glob.glob(f"{result_dir}{modelnames[0]}/*-Angry-0.0-0.wav")
filenames.sort()
for path in tqdm(filenames):
    basename = os.path.basename(path).split(".")[0]
    basename, emotion, _ = basename.split("-")
    worddir_paths[basename] = glob.glob("./../FastSpeech2/preprocessed_data/ESD_spl/" + "ED/" + basename + "*_test_worddir.npy")[0]

100%|██████████| 50/50 [00:44<00:00,  1.12it/s]


In [50]:
def init():
    whole_text = """var TestConfig = {
  "TestName": "Word-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[\n"""
    return whole_text

In [51]:
exid = "main"
num = 3
start = 0
whole_text = init()
result_dir = result_base_dir + f"word-phoneme/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
filenames = glob.glob(f"{result_dir}{modelnames[0]}/*-Angry-0.0-0.wav")
filenames.sort()
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    basename, emotion, _ = basename.split("-")
    if not (basename in evaluation_filenames):
        continue
    spk, name = basename.split("_")
    for mn in modelnames:
        if "msemotts" in mn:
            continue
        modelpath = result_dir + f"{mn}/{basename}-Angry-0.0-0.wav"
        emphasis = np.load(modelpath[:-4]+".npy", allow_pickle=True).item()["effect_w"][0]
        wdpath = worddir_paths[basename]
        word_dir = np.load(wdpath, allow_pickle=True).item()
        words = [a.split("-")[1] for a in list(word_dir.keys())]
        text = ""
        for w, word in enumerate(words):
            add = f"{{{word}}}" if w in emphasis else word
            text += f"{add} "
        text = text[:-1]
        
        starter = "  {\n"
        starter += f'    "Name": "{basename}-{emotion}",\n'
        starter += f'    "TestID": "{mn}---{basename}---{emotion}",\n'
        starter += f'    "Text": "Emotion: {emotion}, Transcript: {text}",\n'
        starter += f'    "Files": {{\n'
        modelpaths = """"""
        for itt, intensity in enumerate(intensities):
            modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
            added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
            modelpaths += added
            
            modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0-cut.wav"
            added = f'      "{intensity_label[itt]}cut": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},\n'
        whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Word-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
]
}


In [53]:
exid = "main"
whole_text = init()
result_dir = result_base_dir + f"word-phoneme/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
test_arrays = []
for basename in evaluation_filenames:
    filenames = glob.glob(f"{result_dir}{modelnames[0]}/{basename}-Angry-0.0-0.wav")
    filenames.sort()
    for path in filenames:
        spk, name = basename.split("_")
        for mn in modelnames:
            if "msemotts" in mn:
                continue
            for emotion in emos:
                test_arrays += [[basename, mn, emotion]]
            
random.shuffle(test_arrays)
for test_array in test_arrays:
    basename, mn, emotion = test_array
    modelpath = result_dir + f"{mn}/{basename}-Angry-0.0-0.wav"
    emphasis = np.load(modelpath[:-4]+".npy", allow_pickle=True).item()["effect_w"][0]
    wdpath = worddir_paths[basename]
    word_dir = np.load(wdpath, allow_pickle=True).item()
    words = [a.split("-")[1] for a in list(word_dir.keys())]
    text = ""
    for w, word in enumerate(words):
        add = f"{{{word}}}" if w in emphasis else word
        text += f"{add} "
    text = text[:-1]

    starter = "  {\n"
    starter += f'    "Name": "{basename}---{emotion}",\n'
    starter += f'    "TestID": "WC---{mn}---{basename}---{emotion}",\n'
    starter += f'    "Text": "Intended Emotion: <b>{emotion}</b>, Transcript: {text}",\n'
    starter += f'    "Files": {{\n'
    modelpaths = """"""
    for itt, intensity in enumerate(intensities):
        modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
        added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
        modelpaths += added

        modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0-cut.wav"
        added = f'      "{intensity_label[itt]}cut": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},\n'
    whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])

var TestConfig = {
  "TestName": "Word-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[
  {
    "Name": "0014_000732---Angry",
    "TestID": "WC---OpenSMILE_EPR-globaloutput-GRL_esd_fcn---0014_000732---Angry",
    "Text": "Intended Emotion: <b>Angry</b>, Transcript: who is been {repeating} all that {hard} {stuff} to you",
    "Files": {
      "L": "./audio/240529_TAC/word-phoneme/OpenSMILE_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.0-0.wav",
      "Lcut": "./audio/240529_TAC/word-phoneme/OpenSMILE_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.0-0-cut.wav",
      "M": "./audio/240529_TAC/word-phoneme/OpenSMILE_EPR-globaloutput-GRL_esd_fcn/0014_000732-Angry-0.6-0.wav",
      "Mcut": "./audio/240529_TAC/word-phoneme/Op